This is one of the deliverables produced from this project: https://library.hkust.edu.hk/ds/project/p002/
> Created by LAU Ming Kit, Jack (Year 4 student, BEng in Computer Engineering, HKUST)

# Demo of using RAM if you want to use the API URLs provided by certain museums or libraries

In [12]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
# Import libraries
import torch

# RAM
from PIL import Image
from ram.models import ram_plus
from ram import inference_ram as inference
from ram import get_transform
from io import BytesIO

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Extract image URLs to "data" dataframe

In [13]:
# https://github.com/hkust-dh/streamlit-demo-20240322/blob/main/pages/2_Import%20Data%20via%20API.py

url='https://lbezone.hkust.edu.hk/rse/OAI/Server.php?verb=ListRecords&metadataPrefix=oai_dc&set=shafei'
response = requests.get(url)
response.encoding = 'utf-8'
xml_content = response.content

# Parse the XML content
root = ET.fromstring(xml_content)

In [14]:
# Create lists to store the extracted information
titles = []
dates = []
languages = []
urls = []
thumb_urls =[]

# Iterate over each record and extract the desired information
for record in root.findall(".//{http://www.openarchives.org/OAI/2.0/}record"):
    title = record.find(".//{http://purl.org/dc/elements/1.1/}title").text
    date = record.find(".//{http://purl.org/dc/elements/1.1/}date").text

    titles.append(title)
    dates.append(date)

    identifiers = record.findall('.//{http://purl.org/dc/elements/1.1/}identifier')
    thumb_url = identifiers[1].text
    thumb_urls.append(thumb_url)

    doi_url = identifiers[4].text
    urls.append(doi_url)

In [16]:
# Create a dataframe from the extracted information
data = {
    "Title": titles,
    "Year": dates,
    "URL": urls,
    "Thumbnail image": thumb_urls
}
data = pd.DataFrame(data)

data.head()

,Title,Year,URL,Thumbnail image
0,Executing collaborator Mayor Han Guangsen and ...,2007,https://doi.org/10.14711/spcol/x001209,https://lbezone.hkust.edu.hk/thumb/thumb-x0012...
1,Eighth Route Army officers explaining the leni...,2007,https://doi.org/10.14711/spcol/x001324,https://lbezone.hkust.edu.hk/thumb/thumb-x0013...
2,Women workers in Jinchaji border area making s...,2007,https://doi.org/10.14711/spcol/x001320,https://lbezone.hkust.edu.hk/thumb/thumb-x0013...
3,Making copperplate for the inaugural issue of ...,2007,https://doi.org/10.14711/spcol/x001322,https://lbezone.hkust.edu.hk/thumb/thumb-x0013...
4,"Sparrow tactics among guerrilla, 1940 (游擊隊麻雀戰,...",2007,https://doi.org/10.14711/spcol/x001306,https://lbezone.hkust.edu.hk/thumb/thumb-x0013...


## Use RAM to the "data" dataframe

In [ ]:
# Prediction
# download the model from the URL below and put in the "pretrained" folder of your working directory 
# https://huggingface.co/xinyu1205/recognize-anything-plus-model/resolve/main/ram_plus_swin_large_14m.pth
ram_pth =  "pretrained/ram_plus_swin_large_14m.pth"
image_size = 384

#######load model
model = ram_plus(pretrained=ram_pth,
                            image_size=image_size,
                            vit='swin_l')
model.eval()

model = model.to(device)

In [ ]:
for i in range(len(data)):
    response = requests.get(data.loc(i, 'URL')) 
    ori_image = Image.open(BytesIO(response.content))

    transform = get_transform(image_size=image_size)

    image = transform(ori_image).unsqueeze(0).to(device)

    res = inference(image, model)
    
    data.loc[i, 'keywords'] = res[0]
data.head()
# data.to_csv('output/websiteAPI.csv')